In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys, gzip, csv
from glob import glob

# core nltk
import nltk
from nltk.tokenize import word_tokenize

# gensim magic
import gensim
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

import numpy as np
import collections

import matplotlib.pyplot as plt
%matplotlib inline

# for PCA
from sklearn.decomposition import PCA

# Linear model
from sklearn.linear_model import LinearRegression

# cosine similarity 
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# load saved models in order
crown_models = list()
input_data = [
    "../models/crown-henry.w2v",
    "../models/crown-edward.w2v",
    "../models/crown-mary.w2v",
    "../models/crown-elizabeth.w2v",
    "../models/crown-james.w2v",
    "../models/crown-charles.w2v"
]    

for fp in input_data:
    model_name = os.path.basename(fp).split(".")[0]
    print("starting: {0}".format(model_name))
    crown_models.append([model_name,KeyedVectors.load(fp,mmap='r')])

starting: crown-henry
starting: crown-edward
starting: crown-mary
starting: crown-elizabeth
starting: crown-james
starting: crown-charles


In [4]:
def get_change_all(model_a, model_b):

    # extract all common vocab
    common_vocab = [word for word in model_a.vocab if word in model_b.vocab]
    common_vectors_a = model_a[common_vocab]
    common_vectors_b = model_b[common_vocab]
    
    # now use Sklearn's LinearRegression to combine vector space
    lin_model = LinearRegression()
    lin_model.fit(common_vectors_a, common_vectors_b)
    
    shared_vectors = collections.OrderedDict()
    
    for word in model_a.vocab:
        word_vector = lin_model.predict(model_a[word].reshape(1, -1))
        shared_vectors[word] = word_vector.reshape(-1)
    
    # now add words only in model b
    #for word in [word for word in model_b.vocab if word not in common_vocab]:
    #    shared_vectors[word] = model_b[word]
        
    vocab = list(shared_vectors.keys())
    shared_embeddings = np.array(list(shared_vectors.values()))
    
    distances = list()
    for word in vocab:
        idx = vocab.index(word)
        distances.append([word,
                          float(cosine_similarity(model_a[word].reshape(1,-1),
                                            shared_embeddings[idx].reshape(1,-1)))])    
    return(distances)

In [5]:
def write_csv(data,model):
    csvfile = gzip.open('../models/' + str(model) + '-drift.csv.gz', 'wt')
    writer = csv.writer(csvfile, delimiter=',')
    for w, d in data:
        writer.writerow([w,d])

In [6]:
# iterate through models
for i, model in enumerate(crown_models):
    if i >= 1:
        print(crown_models[i])
        data = get_change_all(crown_models[i -1][1], crown_models[i][1])
        write_csv(data,crown_models[i][0])

['crown-edward', <gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x1a1bd2b908>]
['crown-mary', <gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x1a1c6142e8>]
['crown-elizabeth', <gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x1a1e080278>]
['crown-james', <gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x1a1e0803c8>]
['crown-charles', <gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x1a1c560eb8>]
